- How to identity contact between buses?
- 100m range --> references
    - Trajectory Improves Data Delivery in Vehicular Networks = 300m
    - Classificando Comportamentos Sociais em Redes Veiculares = 100 to 150m, janelas de agregação --> 5,15,30,60
    - Explorando Dados Urbanos: um estudo usando viagens deT ́axi da cidade de S ̃ao Francisco 100m e 15 min
    - An Alternative Approach to Mobility Analysis inVehicular Ad Hoc Network - 100m
    - CARACTERIZAÇÃO DA MOBILIDADE VEICULAR E O SEU IMPACTO NASREDES VEICULARES TOLERANTES A ATRASOS E DESCONEXÕES - 100m
    - nsights on Metropolitan-Scale VehicularMobility from a Networking Perspectiv - 100m e grafo
    - Generation and Analysis of a Large-Scale UrbanVehicular Mobility Dataset 100m
- Temporal Graph --> references procurar depois


In [1]:
# Getting the minute of the date_avl, and getting the graph_id
# The register will be modeled in a temporal graph where there is a graph for a moment. 
# This graph is composed by nodes which are buses, 
# and edges are connections between buses (nodes) if they are closer from each other less 100 meters

# A graph in the moment t in a region r is composed by register in a interval of 1 minute. Interval examples:
# 10:00 - 10:00:59
# 10:01:00 - 10:01:59
# There will be a graph for each one of the interval above.
# To define which interval the trace belong:
# 1- We get the column hour_avl  (previous calculated) from the date of the register. Ex: 10:30 --> hour_avl = 10
# 2- We get the column minute_avl the minute from the date of the register. Ex: 10:35 --> minute_avl = 35
# 3- We concate the hour_avl,minute_avl,region = graph_id. So, we have graph_id

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1610034250435_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pyspark.sql.functions as F

days_to_analize = [1,4,12,17,20]
days_to_analize_2 = [2,3,5,6,7,8,9,10,11,13,14,15,16,18,19,21,22,23,24,25,26,27,28,29,30,31]


for day in days_to_analize_2:
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    df = traces.repartition(150)
    
    df_graph_id = df\
        .withColumn('minute_avl',F.minute(F.col("dt_avl")))\
        .withColumn('graph_id',F.concat(F.col("hour_avl"),F.lit("-"),F.col("minute_avl"),F.lit("-"),F.col("region")))\
        .drop("hour_diff","dt_avl","speed")

    df_graph_id.repartition("graph_id").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/15-traces-graph-1-minute/MO_1510"+str(day)+"/")
    

In [ ]:
import pyspark.sql.functions as f
days_to_analize = [1,4,12,17,20]
days_to_analize_2 = [2,3,5,6,7,8,9,10,11,13,14,15,16,18,19,21,22,23,24,25,26,27,28,29,30,31]



for day in days_to_analize_2:

    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/15-traces-graph-1-minute/MO_1510"+str(day)+"/")
    df = traces.repartition(150)
    df2 = df
    
    df.alias('df1').join(df2.alias("df2"),on=["graph_id"],how="outer")\
    .select(
        f.col("df1.id_avl").alias("id_avl_1"),
        f.col("df1.line_id").alias("line_1"),
        f.col("df1.latitude").alias("latitude_1"),
        f.col("df1.longitude").alias("longitude_1"),
        f.col("df2.id_avl").alias("id_avl_2"),
        f.col("df2.line_id").alias("line_2"),
        f.col("df2.latitude").alias("latitude_2"),
        f.col("df2.longitude").alias("longitude_2"),
        f.col("df1.hour_avl").alias("hour_avl"),
        f.col("df1.minute_avl").alias("minute_avl"),
        f.col("df1.region").alias("region"),
        f.col("graph_id").alias("graph_id"),
    
).write.parquet("s3://mobility-traces-sp/16-graph-joined-minute/MO_1510"+str(day)+"/")